In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("proj1.ipynb")

# Project 1 - SQL

In this project we will be working with SQL on the IMDB database.

## Due Date

This assignment is due on **Friday, February 19th** at 11:59 pm.

## Scoring Breakdown

|Question|Points|
|---|---|
|0|1|
|1a|1|
|1b|2|
|1c|1|
|2ai|1|
|2aii|3|
|2bi|2|
|2bii|2|
|2biii|2|
|2biv|1|
|3a|2|
|3b|2|
|3c|1|
|4|2|
|**Total**|23|

In [1]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

# Getting Conected
We are going to be using the `ipython-sql` library to connect our notebook to a PostgreSQL database server on your jupyterhub account. The next cell should do the trick; you should not see any error messages after it completes.

In [2]:
# The first time you are running this cell, you may need to run the following line as: %load_ext sql 
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

Now we need to unzip the data.

In [3]:
!unzip -u data/imdbdb.zip -d data/

We will use PostgreSQL commands to create a database and import our data into it. Run the following cell to do this. The line `%sql \l` will display the databases made and you should see a database called `imdb`.

In [4]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS imdb'
!psql -h localhost -c 'CREATE DATABASE imdb' 
!psql -h localhost -c -d imdb -f data/imdbdb.sql
%sql \l

Now let's connect to the new database we just created! There should be no errors after running the following cell.

In [5]:
%sql postgresql://jovyan@127.0.0.1:5432/imdb

To make sure things are working, let's fetch 10 rows from one of our tables.

In [6]:
%%sql
SELECT * 
  FROM cast_sample
 LIMIT 10

## Table Descriptions

We are working with the IMDB database. This database is huge and has a lot of information that we have pared down for this project. 

- actor_sample: information about the actors including id, name, and gender
- cast_sample: each person on the cast of each movie gets a row including cast id, person's id (actor_sample.id), movie id, and role id
- movie_sample: sample of movies the actors have been in including movie's id, title, and the production year
- movie_info_sample: this table originally had a lot of information for each movie (take a look at info_type to see the information available) but we have dropped some information to make it a bit easier to manage. This table includes movie info's id, movie id, info type id, and the info itself
- info_type: reference table to match each info type id to the description of the type of information
- role_type: reference table for cast_sample to match role id to the description of the role

To make these tables smaller in your snapshot of IMDB, we took a random sample of actors from the full database, and included their corresponding movies and cast info. You will learn how to do sampling in SQL below.

Let's look at metadata about the tables. Many database clilent applications (like dbeaver or psql) and connectivity libraries offer some convenience commands for exploring metadata. We can use the psql's meta-commands (also called "backslash commands") from Jupyter directly.

In [7]:
%sql \d

You can get quick help on psql meta-commands via \?:

In [8]:
%sql \?

There is a more "native SQL" way to look at metadata. SQL stores its metadata in tables -- essentially metadata is just data! So we can use SQL to query the metadata. We access metadata table names via a prefix information_schema.. So for example we can query the table of all tables as information_schema.tables. We want to restrict it to tables in our default schema, "public", so we use a WHERE clause:

In [9]:
%%sql
SELECT * 
FROM information_schema.tables
WHERE table_schema = 'public';

## Question 0
Now you write a query to get the name of all the tables in the PostgreSQL information_schema schema! 

Note: The `result_0 <<` syntax stores the result of the SQL query to the variable `result_0`.

<!--
BEGIN QUESTION
name: q0
points: 1
-->

In [10]:
%%sql result_0 <<
...

In [ ]:
grader.check("q0")

# Question 1: Assessing Table Contents 
One of the first things you'll want to do with a database table is get a sense for its metadata: column names and types, and number of rows. 

We can use the PostgreSQL `\d` meta-command to get a description of all the columns in the `movie_info_sample` table.

In [14]:
%sql \d movie_info_sample

## Question 1a
In the next cell, write an SQL query to calculate the number of rows in the `movie_info_sample` table.

<!--
BEGIN QUESTION
name: q1a
points: 1
-->

In [15]:
count_1a = ...
count_1a

In [ ]:
grader.check("q1a")

### Taking a Random Sample (and Python variable substitution)

The cell positioned 2 below this one shows you a good way to take a random sample from a table: use the `TABLESAMPLE` clause after the table name in the `FROM` clause. It also shows you how you can reference a Python variable within a SQL string in the connection library we're using, `ipython-sql`. (The syntax for language-variable substitution is not part of the SQL standard, it's done by the connection library, so this syntax is specific to `ipython-sql`)

The SQL query below fetches a sample of size `p%` via the [Bernoulli sampling](https://wiki.postgresql.org/wiki/TABLESAMPLE_Implementation#BERNOULLI_Option) scheme, which instructs the database engine to "flip a coin" before deciding whether to return each tuple. 

## Question 1b
Given that you know the size of the table from the previous query, choose a sampling rate `p` that retrieves 5 tuples on expectation. Don't forget to express `p` in units of percent (i.e. `p=5` is 5%)!

Try running the SQL cell many times and see what you notice.

<!--
BEGIN QUESTION
name: q1b
points: 2
-->

In [18]:
p_1b = ...
p_1b

In [ ]:
grader.check("q1b")

In [21]:
%%sql
SELECT *
  FROM movie_info_sample TABLESAMPLE bernoulli('{p_1b}')

## Question 1c

If you don't care about randomness, a more efficient way to get some arbitrary example rows from a table is to just use a `LIMIT` clause. In the next cell, fetch 5 rows from `movie_info_sample` using `LIMIT`.

<!--
BEGIN QUESTION
name: q1c
points: 1
-->

In [22]:
%%sql result_1c <<
...

In [ ]:
grader.check("q1c")

# Question 2: Data Cleaning and Transformation

## Question 2a: MPAA Rating

The MPAA rating is something most datasets about movies contain and ours is no exception! But it's pretty messy to extract from the existing data. We're going to create a nice refined view of the `movie_sample` table that also includes a rating field.

### Question 2ai:
To start, using the `info_type` table, find which `info_type_id` corresponds to a film's MPAA rating. Store the resulting tuple from the `info_type` table in results.

<!--
BEGIN QUESTION
name: q2ai
points: 1
-->

In [24]:
mpaa_rating_id = ...
mpaa_rating_id

In [ ]:
grader.check("q2ai")

You will want to reuse the query above via copy/paste in the next cell or use `mpaa_rating_id` directly using python variable substitution.

### Question 2aii:
In the next cell please construct a view named `movie_ratings` containing one row for each movie containing `(movie_id, title, info, mpaa_rating)`, where `info` is the full MPAA rating string from `movie_info_sample`, and `mpaa_rating` is just the rating itself (i.e. `R`, `PG-13`, `PG`, etc). Following the view definition, write a `SELECT` query to return the first 20 rows of the view ordered by ascending `movie_id`.

Before you get started, take a look at the `movie_info_sample` tuples corresponding to the MPAA rating. The `info` field is a little longer than just the rating. It includes an explanation for why that movie received its score. You will need to extract a substring from the `info` column of `movie_info_sample`; you can use the [string functions](https://www.postgresql.org/docs/current/functions-string.html) in PostgreSQL to do it. There are many possible solutions. You might choose to do it by position (STRPOS), by using the SUBSTRING function, or both. We recommend you define a CTE (`WITH` clause) that does this part, and then use it to complete the query.

<!--
BEGIN QUESTION
name: q2aii
points: 3
-->

In [26]:
%%sql result_2aii <<

DROP VIEW IF EXISTS movie_ratings;
CREATE VIEW movie_ratings AS

...

In [ ]:
grader.check("q2aii")

## Question 2b - Movie Moola

One measure of a movie's success is how much money it makes. If we look at our `info_type` table, we have information about the film's gross earnings and the budget for a film. It would be nice to know how much money a film made using the profit formula:
$$profit = earnings - moneyspent$$

We start by taking a look at the gross info types.

In [30]:
%%sql
SELECT * 
FROM movie_info_sample
WHERE info_type_id = 107
Order BY id
LIMIT 10 OFFSET 100000;

There are a lot of things to notice here. First of all, the `info` is a string with not only the earnings but the country and the month the earnings are cummulatively summed till. Additionally, the info is not in the same currency! On top of that, it looks like some of the gross earnings even for those in USD are from worldwide sales while others only count sales within the USA.

For consistency, let's only use movies with gross earnings counted in the USA.

### Question 2bi:
We want the number part of the `info` column and the latest (maximum) earnings value for a particular film. Clean `info` for the gross earnings information and extract the dollar amount as an float. The resulting table should have columns for the `gross` as a float, `movie_id`, and `title`. 

We are going to need this table later on when calucting the movie's profit so let's save the query as a view called `movie_gross`.

To take a look at our cleaned data, display the `title` for the top 10 highest grossing films along with their `movie_id` and `gross`.

- HINT: We extracted the rating above just like how we want to isolate the dollar amount as a string (There are multiple ways of doing this).

- HINT: Look at the [regexp_replace](https://www.postgresql.org/docs/9.4/functions-matching.html) function and the 'g' tag

<!--
BEGIN QUESTION
name: q2bi
points: 2
-->

In [31]:
%%sql result_2bi <<

DROP VIEW IF EXISTS movie_gross;
CREATE VIEW movie_gross AS

...

In [ ]:
grader.check("q2bi")

### Question 2bii:

Now, we want something similar for the budget of the film so we can perform the subtraction of the `gross` and `budget`. Clean `info` for the budget information and extract the dollar amount as an float. The resulting table should have columns for the `budget` as a float, `movie_id`, and `title`. 

We are going to need this table later on as well when calculating the movie's profit so let's save the query as a view called `movie_budget`.

To take a look at our cleaned data, display the `title` for the top 10 highest budget films along with their `movie_id` and `budget`.

<!--
BEGIN QUESTION
name: q2bii
points: 2
-->

In [35]:
%%sql result_2bii <<

DROP VIEW IF EXISTS movie_budget;
CREATE VIEW movie_budget AS

...

In [ ]:
grader.check("q2bii")

### Question 2biii:

We have all the parts we need to calculate the profits. Using the `movie_gross` and `movie_budget` views created above, we can subtract the numeric columns and save the reuslt in another column called `profit`.

In the next cell, construct a view named `movie_profit` containing one row for each movie containing `(movie_id, title, profit)`, where `profit` is the result of subtracting that movie's `budget` from `gross`. Following the view definition, write a `SELECT` query to return the first 20 rows of the view ordered by descending `profit`. This may take a second.

<!--
BEGIN QUESTION
name: q2biii
points: 2
-->

In [39]:
%%sql result_2biii <<

DROP VIEW IF EXISTS movie_profit;

CREATE VIEW movie_profit AS

...

In [ ]:
grader.check("q2biii")

<!-- BEGIN QUESTION -->

### Question 2biv:

We analyzed the data but something seems odd. On a closer look, there are many negative values for `profit`. For example, the movie `102 Dalmations` looks to have lost ~$18M but it was a widely successful film! What may account for this issue?

<!--
BEGIN QUESTION
name: q2biv
manual: true
points: 1
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



# Question 3: Using Cleaned Data

Now that we have gone through all the work of cleaning our financials from `info` in `movie_info_sample`, let's take a closer look at the data we generated. 

### Question 3a: Earnings per Genre

Another `info_type` available to us is the movie genre. Looking at the `movie_gross` values, how much does each genre earn on average from the US?

Create a view with the columns `movie_id`, `title`, `gross`, `genre`, and `average_genre` where `gross` is a movie's gross US earnings, `genre` is the movie's genre, and `average_genre` is the average earnings for the corresponding genre. If a movie has multiple genres, the movie should appear in multiple rows with each genre as a row.

Following the view definition, write a `SELECT` query to return the rows for the movie "Mr. & Mrs. Smith" ordered by genre alphabetically.

- HINT: Look into [window functions](https://www.postgresql.org/docs/9.1/tutorial-window.html)

<!--
BEGIN QUESTION
name: q3a
points: 2
-->

In [43]:
%%sql result_3a <<

DROP VIEW IF EXISTS movie_avg_genre;
CREATE VIEW movie_avg_genre AS

...

In [ ]:
grader.check("q3a")

### Question 3b: Analyzing Gross Earnings

A good way to view numerical data is through a histogram. The histogram shows the spread of the data along with several other key attributes that allow for the data to be analyzed. 

We went through a lot of work transforming the gross earnings `info` string into a numerical value. Because of our hard work, we can now look more closely at this data and understand what it looks like. We will generate a [five-number summary](https://en.wikipedia.org/wiki/Five-number_summary) and the average of the US gross earnings data to do this.

Create a view of a one row summary of the `movie_gross` `gross` data with the `min`, `25th_percentile`, `median`, `75th_percentile`, `max`, and `average`. Following the view definition, display it.

- HINT: Look at SQL [aggregate functions](https://www.postgresql.org/docs/9.4/functions-aggregate.html). You may find some useful.

<!--
BEGIN QUESTION
name: q3b
points: 2
-->

In [47]:
%%sql result_3b <<

DROP VIEW IF EXISTS earnings_summary;
CREATE VIEW earnings_summary AS

...

In [ ]:
grader.check("q3b")

<!-- BEGIN QUESTION -->

### Question 3c

What do you notice about the summary values generated in `earnings_summary`? Identify two properties about the histogram of the data. 
- HINT: Think in terms of about concepts from statistics like spread, modality, skew, etc. and hwo they may apply here

<!--
BEGIN QUESTION
name: q3c
manual: true
points: 1
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

# Question 4: Joins

Joins are a powerful tool in database cleaning. They allow for the user to create useful tables and bring together information in a meaningful way. 

There are many types of joins: inner, outer, left, right, etc. Let's practice these in a special scenario. 

You are now working as a talent director and you need a list of all `actor` roles and the number of movies in which they have acted. 

Create a view called `number_movies` which columns `id`, `name`, `number` where `id` is the actor's id, `name` is the actor's name, and `number` is the number of movies they have acted in. 

NOTE: The `cast_sample` may include actors not included in `actor_sample` table. We still want to include these actors in our result by reference to their id. The `name` field can be NULL.

Following your view, display the `id`, `name`, and `number` of films for the top 10 actors who have been in the most films.

<!--
BEGIN QUESTION
name: q4
points: 2
-->

In [51]:
%%sql result_4 <<

DROP VIEW IF EXISTS number_movies;
CREATE VIEW number_movies AS 

...

In [ ]:
grader.check("q4")

# DONE!

Great job! You finished your first project in Data Engineering. To submit, please run the `grader.check_all()` cell below and save your notebook. After that, submit your `proj1.ipynb` file to Gradescope. Make sure that we can see the results of your `grader.check_all()` cell on your Gradescope submission. That's all!

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()